In [1]:
import numpy as np

def preprocess(filename):
    ret = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.lower()
            last_space = True
            new_line = []
            for i in line:
                if i.isalpha():
                    v = ord(i)-ord('a')+1
#                     new_vector = np.zeros(27)
#                     new_vector[v] = 1.0
                    new_line += [[v]]
                    last_space = False
                else:
                    if last_space == False:
#                         new_vector = np.zeros(27)
#                         new_vector[26] = 1.0
                        new_line += [[27]]
                        last_space = True
            ret += [new_line]
    return ret

jp_x = preprocess('jp.txt')
en_x = preprocess('en.txt')
x = jp_x + en_x
y = [[1.0,0.0] for i in jp_x] + [[0.0,1.0] for i in en_x]


In [3]:
def align(data):
    max_len = 0
    for row in data:
        if len(row) > max_len:
            max_len = len(row)
    ret = []
    for row in data:
        ret += [row + [[0]]*(max_len-len(row))]
    return ret,max_len

x,seq_max_len = align(x)

In [6]:
import random
r_index = list(range(len(x)))
random.shuffle(r_index)
train_x = [x[i] for i in r_index[:int(len(r_index)*0.8)]]
train_y = [y[i] for i in r_index[:int(len(r_index)*0.8)]]
test_x = [x[i] for i in r_index[int(len(r_index)*0.8):]]
test_y = [y[i] for i in r_index[int(len(r_index)*0.8):]]

In [9]:
print(len(train_x))
print(len(train_y))
print(seq_max_len)
print(train_y)

529
529
75
[[1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0

In [11]:
import tensorflow as tf
print(tf.__version__)

1.3.0


In [26]:
a = tf.unstack([[1,2,1],[1,1,2]], 3, 1)
with tf.Session() as session:
    print(a[0].eval())
    print(a[1].eval())
    print(a[2].eval())

[1 1]
[2 1]
[1 2]


In [50]:
n_hidden = 50
num_steps = 5000
n_classes = 2

def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

graph = tf.Graph()
with graph.as_default(), tf.device('/gpu:0'):
    
    weights = {
        'state' : tf.Variable(tf.random_normal([50, 2])),
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    
    def dynamicRNN(x, weights, biases):
        seqlen = length(x)
        x = tf.unstack(x, seq_max_len, 1)
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)
        outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                    sequence_length=seqlen)

        # When performing dynamic calculation, we must retrieve the last
        # dynamically computed output, i.e., if a sequence length is 10, we need
        # to retrieve the 10th output.
        # However TensorFlow doesn't support advanced indexing yet, so we build
        # a custom op that for each sample in batch size, get its length and
        # get the corresponding relevant output.

        # 'outputs' is a list of output at every timestep, we pack them in a Tensor
        # and change back dimension to [batch_size, n_step, n_input]
        outputs = tf.stack(outputs)
        outputs = tf.transpose(outputs, [1, 0, 2])

        # Hack to build the indexing and retrieve the right output.
        batch_size = tf.shape(outputs)[0]
        # Start indices for each sample
        index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
        # Indexing
        outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)
        
        
        states = tf.transpose(states, [1, 2, 0])
        
        states = states*weights['state']

        # Linear activation, using outputs computed above
        return states* weights['out'] + biases['out']
        
    x = tf.placeholder("float", [None, seq_max_len,1])
    y = tf.placeholder("float", [None, n_classes])
    inputs = x
    labels = y
    logits = dynamicRNN(x, weights, biases)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    optimizer = tf.train.AdagradOptimizer(learning_rate = 0.5).minimize(loss)
    
    correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    feed_dict = {inputs: train_x, labels: train_y}
    _, l, predictions, acc = session.run([optimizer, loss, logits, accuracy],feed_dict = feed_dict)
    if (step % 100 == 0):
      print('Loss at step %d: %f acc: %f' % (step, l,acc))
#       print predictions[:10]
#       print train_y[:10]
  print("Testing Accuracy:", \
        session.run(accuracy, feed_dict={inputs: test_x, labels: test_y}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized


InvalidArgumentError: Incompatible shapes: [529,2] vs. [529]
	 [[Node: Equal = Equal[T=DT_INT64, _device="/job:localhost/replica:0/task:0/gpu:0"](ArgMax, ArgMax_1)]]
	 [[Node: Mean_1/_319 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_30793_Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Equal', defined at:
  File "/home/lorabit/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/lorabit/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/lorabit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/lorabit/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File